#### Стажировочная работа

_Дябкин Ярослав сергеевич_

## Введение и постановка задачи

На сегодняшний день задача обнаружения мошеннических аккаунтов является очень важной для обеспечения безопасности онлайн-торговли.

В предложенном наборе данных представлена информация о различных объявлениях пользователей.

Задача - определить, является ли пользователь мошенником.

### Описание входных данных

Файлы
Набор данных состоит из 3 файлов.

train.csv - данные, используемые для обучения моделей.
test.csv - данные, используемые для получения предсказаний.
sample_submission.csv - пример файла ответа с требуемой структурой.

Столбцы
profile - идентификатор пользователя
location - город пользователя
score - рейтинг пользователя
reviews - количество отзывов на пользователя
counted - количество оценок
item_id - идентификатор объявления
item_name - название объявления
price - цена, указанная в объявлении
category_id - категория объявления
status - является ли мошенническим аккаунт (0 - не является, 1 - является)

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
# Загрузка данных
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [3]:
# Удалим ненужные столбцы
drop_cols = ['profile', 'item_id', 'item_name']
X = train.drop(columns=drop_cols + ['status'])
X_test = test.drop(columns=drop_cols)
y = train['status']
profiles_test = test['profile']


In [4]:
# Объединяем X и X_test для единой обработки
all_data = pd.concat([X, X_test], axis=0)

# Кодируем категориальные признаки
le_dict = {}
for col in all_data.columns:
    if all_data[col].dtype == 'object':
        le = LabelEncoder()
        all_data[col] = le.fit_transform(all_data[col].astype(str))
        le_dict[col] = le

# Разделим обратно
X = all_data.iloc[:len(X)]
X_test = all_data.iloc[len(X):]


In [5]:
# Обучение и сравнение нескольких моделей
models = {
    'Random Forest': RandomForestClassifier(n_estimators=1000, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'SVC': SVC()
}


In [6]:
print("\nОценка моделей по кросс-валидации:")


Оценка моделей по кросс-валидации:


In [7]:
best_model_name = None
best_score = 0
best_model = None

for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    mean_score = scores.mean()
    print(f"{name}: {mean_score:.4f}")
    if mean_score > best_score:
        best_score = mean_score
        best_model_name = name
        best_model = model

print(f"\nЛучшая модель: {best_model_name} (точность: {best_score:.4f})")

Random Forest: 0.8935
Logistic Regression: 0.8294
SVC: 0.7710

Лучшая модель: Random Forest (точность: 0.8935)


In [8]:
# Обучим лучшую модель на всех данных
best_model.fit(X, y)

# Предсказание на тестовой выборке
preds = best_model.predict(X_test)

In [9]:
# Сохраняем submission
submission = pd.DataFrame({
    'profile': profiles_test,
    'status': preds
})

submission.to_csv('submission.csv', index=False)
print("\nФайл 'submission.csv' успешно сохранён.")


Файл 'submission.csv' успешно сохранён.


В ходе выполнения работы были обучены и сравнены три модели машинного обучения: Random Forest, Logistic Regression и Support Vector Classifier (SVC). Оценка производилась с помощью кросс-валидации по метрике accuracy:

Оценка моделей по кросс-валидации:
Random Forest: 0.8935
Logistic Regression: 0.8294
SVC: 0.7710

Наилучший результат показала модель Random Forest с точностью 0.8935, после чего она была обучена на всех доступных обучающих данных.

Файл с предсказаниями (submission.csv) был успешно сформирован и отправлен для проверки на платформу Kaggle, используемую в рамках прохождения курса.
По результатам проверки на сайте итоговая точность составила 0.951, что подтверждает высокое качество модели на тестовых данных.
